In [1]:
import pylhe
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
l1 = 1.5*0.01 + 5*0.001
l2 = 15.0*0.01 - 5*0.001
m_e = 0.000511
m_mu = 0.105
luminosity = 1.5e+04
ee = np.sqrt(4*np.pi/137.)
xsec = 3.488e+07

In [4]:
lhefile = "/Users/isaac/Work/MUonE-displaced-vertex/SM2to4gamma/Events/run_01/unweighted_events.lhe"
four_momentum = np.zeros((10000,4,4), dtype="float64")

In [21]:
for ievent, event in enumerate(pylhe.readLHE(lhefile)):
    for particle in event.particles:
        if particle.id == 13 and particle.status == 1:
            four_momentum[ievent,0] = np.array([particle.e, particle.px, particle.py, particle.pz])
        if particle.id == -11:
            four_momentum[ievent,3] = np.array([particle.e, particle.px, particle.py, particle.pz]) 
        if particle.id == 11 and particle.status == 1:
            four_momentum[ievent,1:3] = np.array([particle.e, particle.px, particle.py, particle.pz])

In [23]:
conv_data = np.zeros((10000,18), dtype="float64")
# muon
conv_data[:,0] = four_momentum[:,0,0] # muon energy
conv_data[:,1] = np.linalg.norm(four_momentum[:,0,1:4], axis=1) # muon p
conv_data[:,2] = np.arccos(four_momentum[:,0,3]/conv_data[:,1]) # muon theta
conv_data[:,3] = np.arctan2(four_momentum[:,0,2], four_momentum[:,0,1]) # muon phi

# electron 1
conv_data[:,4] = four_momentum[:,1,0] # electron 1 energy
conv_data[:,5] = np.linalg.norm(four_momentum[:,1,1:4], axis=1) # electron 1 p
conv_data[:,6] = np.arccos(four_momentum[:,1,3]/conv_data[:,5]) # electron 1 theta
conv_data[:,7] = np.arctan2(four_momentum[:,1,2], four_momentum[:,1,1]) # electron 1 phi

# electron 2
conv_data[:,8] = four_momentum[:,2,0] # electron 2 energy
conv_data[:,9] = np.linalg.norm(four_momentum[:,2,1:4], axis=1) # electron 2 p
conv_data[:,10] = np.arccos(four_momentum[:,2,3]/conv_data[:,9]) # electron 2 theta
conv_data[:,11] = np.arctan2(four_momentum[:,2,2], four_momentum[:,2,1]) # electron 2 phi

# positron
conv_data[:,12] = four_momentum[:,3,0] # positron energy
conv_data[:,13] = np.linalg.norm(four_momentum[:,3,1:4], axis=1) # positron p
conv_data[:,14] = np.arccos(four_momentum[:,3,3]/conv_data[:,13]) # positron theta
conv_data[:,15] = np.arctan2(four_momentum[:,3,2], four_momentum[:,3,1]) # positron phi

# angle between positron and electron
conv_data[:,16] = np.arccos(np.einsum("ij,ij->i", four_momentum[:,1,1:4], four_momentum[:,3,1:4])/(conv_data[:,5]*conv_data[:,13])) # positron and electron 1
conv_data[:,17] = np.arccos(np.einsum("ij,ij->i", four_momentum[:,2,1:4],four_momentum[:,3,1:4])/(conv_data[:,9]*conv_data[:,13])) # positron and electron 2

## Cuts
- 5 GeV lowest energy
- angular acceptance

In [46]:
selection = ((conv_data[:,0] >=5.)& (conv_data[:,4] >= 5.)& (conv_data[:,8]>=5.)& (conv_data[:,12]>=5.)& (conv_data[:,2]<=0.05)& (conv_data[:,6] <= 0.05)&(conv_data[:,10]<=0.05)& (conv_data[:,14]<=0.05)&(conv_data[:,16] >= 0.001)& (conv_data[:,17]>=0.001))
passnumber = conv_data[selection]

Use the tracking layers:

In [40]:
(len(passnumber)/10000.0)*luminosity*xsec*2.85e-07 # The last number is the probability corresponds to the 5 sigma

1342.008

In [47]:
(len(passnumber)/10000.0)

0.009

If forcing to enter the ECAL, only keep the last 5 modules

In [42]:
selection = []
for i in range(0,5):
    selection.append(((conv_data[:,0] >=5.)& (conv_data[:,4] >= 5.)& (conv_data[:,8]>=5.)& (conv_data[:,12]>=5.)& (conv_data[:,2]<=0.05/float(i+1))& (conv_data[:,6] <= 0.05/float(i+1))&(conv_data[:,10]<=0.05/float(i+1))& (conv_data[:,14]<=0.05/float(i+1))&(conv_data[:,16] >= 0.001)& (conv_data[:,17]>=0.001)))


In [44]:
sum([(len(conv_data[selection[i]])/10000.0) for i in range(0,5)])/5 *(luminosity*5./40.)*xsec*2.85e-07

165.8871

In [45]:
sum([(len(conv_data[selection[i]])/10000.0) for i in range(0,5)])/5

0.0089